In [7]:
from pymongo import MongoClient

# Inisialisasi client dan koneksi ke database
client = MongoClient('mongodb://localhost:27017/')
db = client['company_db']
collection = db['employees']

# Contoh untuk memastikan koneksi
print('Connected to MongoDB')

Connected to MongoDB


In [3]:
# Create
employee_data = {'name': 'Alice', 'department': 'Finance', 'age': 29, 'salary': 4500}
collection.insert_one(employee_data)

# Tambahan data
collection.insert_many([
    {'name': 'Bob', 'department': 'IT', 'age': 31, 'salary': 6000},
    {'name': 'Diana', 'department': 'IT', 'age': 26, 'salary': 6200},
    {'name': 'Eva', 'department': 'Finance', 'age': 35, 'salary': 7000}
])

# Update
collection.update_one({'name': 'Alice'}, {'$set': {'salary': 5000}})

# Delete
collection.delete_one({'name': 'Bob'})

# Read
for doc in collection.find():
    print(doc)

{'_id': ObjectId('6902e934f39e86fa98610682'), 'name': 'Alice', 'department': 'Finance', 'age': 29, 'salary': 5000}
{'_id': ObjectId('6902e935f39e86fa98610684'), 'name': 'Diana', 'department': 'IT', 'age': 26, 'salary': 6200}
{'_id': ObjectId('6902e935f39e86fa98610685'), 'name': 'Eva', 'department': 'Finance', 'age': 35, 'salary': 7000}


In [4]:
pipeline = [
    {'$group': {'_id': '$department', 'average_salary': {'$avg': '$salary'}}}
]
for result in collection.aggregate(pipeline):
    print(result)

{'_id': 'IT', 'average_salary': 6200.0}
{'_id': 'Finance', 'average_salary': 6000.0}


In [10]:
products = db['products']
data_produk = [
    {'name': 'Laptop', 'category': 'Electronics', 'price': 1200},
    {'name': 'Chair', 'category': 'Furniture', 'price': 200},
    {'name': 'Cup', 'category': 'Kitchen', 'price': 15},
    {'name': 'Table', 'category': 'Furniture', 'price': 350},
    {'name': 'Monitor', 'category': 'Electronics', 'price': 300}
]
products.insert_many(data_produk)

InsertManyResult([ObjectId('69095e414f7ccb03c2262bfd'), ObjectId('69095e414f7ccb03c2262bfe'), ObjectId('69095e414f7ccb03c2262bff'), ObjectId('69095e414f7ccb03c2262c00'), ObjectId('69095e414f7ccb03c2262c01')], acknowledged=True)

In [6]:
avg_price = list(products.aggregate([
    {'$group': {'_id': None, 'avg_price': {'$avg': '$price'}}}
]))[0]['avg_price']

for p in products.find({'price': {'$gt': avg_price}}):
    print(p)

{'_id': ObjectId('6902edabf39e86fa98610686'), 'name': 'Laptop', 'category': 'Electronics', 'price': 1200}


In [7]:
for result in products.aggregate([
    {'$group': {'_id': '$category', 'total_products': {'$sum': 1}}}
]):
    print(result)

{'_id': 'Kitchen', 'total_products': 1}
{'_id': 'Electronics', 'total_products': 2}
{'_id': 'Furniture', 'total_products': 2}


In [11]:
pipeline = [
    {'$sort': {'salary': -1}},
    {'$group': {
        '_id': '$department',
        'top_employees': {'$push': {'name': '$name', 'salary': '$salary'}}
    }},
    {'$project': {'top_employees': {'$slice': ['$top_employees', 5]}}}
]

for result in collection.aggregate(pipeline):
    print(result)

{'_id': 'Finance', 'top_employees': [{'name': 'Eva', 'salary': 7000}, {'name': 'Alice', 'salary': 5000}]}
{'_id': 'IT', 'top_employees': [{'name': 'Diana', 'salary': 6200}]}


In [12]:
delete_result = collection.delete_many({'age': {'$lt': 25}})
print("Jumlah karyawan dihapus:", delete_result.deleted_count)

Jumlah karyawan dihapus: 0


In [13]:
pipeline = [
    {'$group': {
        '_id': '$department',
        'total_salary': {'$sum': '$salary'},
        'avg_age': {'$avg': '$age'}
    }}
]

for result in collection.aggregate(pipeline):
    print(result)

{'_id': 'Finance', 'total_salary': 12000, 'avg_age': 32.0}
{'_id': 'IT', 'total_salary': 6200, 'avg_age': 26.0}
